<a href="https://colab.research.google.com/github/ailab-nda/ML/blob/main/FSML_Python/chap03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 3. 識別 ー概念学習ー

## カテゴリカルデータで決定木を作成

ライブラリの読み込み

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier, plot_tree

Contact-lensesデータの取得

In [ ]:
df = fetch_openml(name='contact-lenses', parser="auto")
df.data

In [ ]:
X = df.data.drop('contact-lenses', axis=1)
y = df.data.loc[:, 'contact-lenses']

 Contact-lensesデータの特徴値とクラス

 - age (目の年齢) {'young', 'pre-presbyopic', 'presbyopic'} (若年, 老眼前期, 老眼)
 - spectacle-prescrip (眼鏡) {'myope', 'hypermetrope'} (近視, 遠視)
 - astigmatism (乱視) {'no', 'yes'} (なし, あり)
 - tear-prod-rate (涙量) {'reduced', 'normal'} (減少, 正常)
 - contact-lenses (クラス) {'soft', 'hard', 'none'} (ソフト, ハード, なし)

カテゴリカルデータを数値データに変換


*   特徴ベクトルは[OneHotEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html)で、各特徴の値毎のバイナリ値に変換
*   正解は[LabelEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html)で整数値に変換



In [ ]:
enc_X =  OneHotEncoder(sparse_output=False)
X2 = enc_X.fit_transform(X)
enc_X.get_feature_names_out()

In [ ]:
X2

In [ ]:
enc_y = LabelEncoder()
y2 = enc_y.fit_transform(y)
enc_y.classes_

In [ ]:
y2

決定木の学習と表示


*   ノードに書かれている分岐条件が成立するならば左、不成立ならば右の枝を選ぶ
  * 左の枝に True, 右の枝に False のラベルが付いていると解釈すればよい
  * たとえば `tear-prod-rate_normal <= 0.5` は `tear-prod-rate` の値が `normal` でなければ（すなわち`reduced`であれば）、左の枝を選ぶと解釈する
* ノードのclassは最頻のクラスを表す
* リーフのclassは分類結果を表す
* いずれも色で（最頻の）クラス、色の濃さでその割合を表す



木の表示は、scikit-learn ver0.21以降で導入された [sklearn.tree.plot_tree](https://scikit-learn.org/stable/modules/generated/sklearn.tree.plot_tree.html) メソッドによって、教科書p.57のコードよりもかなり簡単になっています。  

In [ ]:
clf = DecisionTreeClassifier(min_samples_leaf=3, random_state=9)
clf.fit(X2, y2)
plt.figure(figsize=(10,7)) #表示領域を(横, 縦)で指定
plot_tree(clf, filled=True, fontsize=10,
          feature_names=enc_X.get_feature_names_out(),
          class_names=enc_y.classes_)
plt.show()

## 数値データで決定木を作成

第2章で用いたirisデータで決定木を作成します。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import cross_val_score

In [ ]:
iris = load_iris()
X = iris.data
y = iris.target

まず、全学習データで決定木を作成し、表示します。

In [ ]:
clf = DecisionTreeClassifier()
clf.fit(X, y)

In [ ]:
plt.figure(figsize=(14,10))
plot_tree(clf, filled=True, fontsize=10, feature_names=iris.feature_names, class_names=iris.target_names)
plt.show()

交差確認法で性能評価を行います。

In [ ]:
scores = cross_val_score(clf, X, y, cv=10)
print(f'Accuracy: {scores.mean():.2f} (+/- {scores.std()*2:.2f})')

# 演習問題

scikit-learn付属のwineデータで決定木の作成・表示と交差確認法による評価を行ってください。可能ならば、木の深さを変えて性能の違いを調べてください。（wineデータの説明 -> https://atmarkit.itmedia.co.jp/ait/articles/2208/25/news046.html）


## 解答欄

決定木の作成および表示

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_wine
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import cross_val_score

In [ ]:
wine = load_wine(as_frame=True)
X = wine.data
y = wine.target

In [ ]:
X.describe()

木の作成

In [ ]:
# 決定木を作成し、
# 学習を行い、
# 結果をプロットするプログラムをここに書く

木の深さと性能の関係

In [ ]:
clf = DecisionTreeClassifier(max_depth=2)
scores = cross_val_score(clf, X, y, cv=10)
print(f'Accuracy: {scores.mean():.2f} (+/- {scores.std()*2:.2f})')

## 3. 解答例

決定木では、特徴ベクトルの個々の次元の値のみでデータの分割が行われるので、他の次元とスケールや平均が異なっていても、結果に影響を及ぼしません。また、標準化を行って値そのものを変えてしまうと、学習された決定木の解釈性が失われます。

## 参考 find-S アルゴリズム

In [ ]:
# generated by GPT-4

import numpy as np
import pandas as pd
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import LabelEncoder

# contact-lenses データの取得
df = fetch_openml(name='contact-lenses', parser="auto")
X = df.data.drop('contact-lenses', axis=1)
y = df.data.loc[:, 'contact-lenses']

# データのエンコード
# 特徴量の名前を取得してエンコーディングする
feature_names = X.columns
X_encoded = X.copy()
encoders = {}

for col in feature_names:
    # 各特徴量のエンコーダーを生成
    encoders[col] = LabelEncoder()
    X_encoded[col] = encoders[col].fit_transform(X[col])

# ターゲット変数のエンコーディング
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# noneの値をエンコードする
target_value = le.transform(['none'])[0]

# find-sアルゴリズムの実装
def find_s(data, target, target_value):
    # 最初の正の例を見つける
    for i in range(len(target)):
        if target[i] == target_value:
            specific_hypothesis = data.iloc[i].tolist()
            break

    # 最も特定的な仮説を見つける
    for i in range(len(target)):
        if target[i] == target_value:
            instance = data.iloc[i].tolist()
            specific_hypothesis = [specific_hypothesis[j] if specific_hypothesis[j] == instance[j] else '?' for j in range(len(specific_hypothesis))]
            print(f'{i}: {specific_hypothesis}')

    return specific_hypothesis

# アルゴリズムを適用
hypothesis = find_s(X_encoded, y_encoded, target_value)

# 結果の表示
print("Find-Sの結果の仮説:")
print(hypothesis)

# 各特徴量のデコーディング（結果の仮説を理解しやすい形にする）
decoded_hypothesis = {}
for i, feature in enumerate(feature_names):
    if hypothesis[i] != '?':
        decoded_hypothesis[feature] = encoders[feature].inverse_transform([hypothesis[i]])[0]
    else:
        decoded_hypothesis[feature] = '?'

print("\nデコードされた仮説:")
print(decoded_hypothesis)
